In [1]:
import numpy as np
import tensorflow as tf
from getEmbeddings import getEmbeddings
import matplotlib.pyplot as plt
import scikitplot.plotters as skplt
import pickle
import os.path

IN_DIM = 300
CLASS_NUM = 2
LEARN_RATE = 0.0001
TRAIN_STEP = 20000
tensorflow_tmp = "tmp_tensorflow/three_layer2"


def plot_cmat(yte, ypred):
    '''Plotting confusion matrix'''
    skplt.plot_confusion_matrix(yte,ypred)
    plt.show()


def dummy_input_fn():
    return np.array([1.0] * IN_DIM)


def model_fn(features, labels, mode):
    """The model function for tf.Estimator"""
    # Input layer
    input_layer = tf.reshape(features["x"], [-1, IN_DIM])
    # Dense layer1
    dense1 = tf.layers.dense(inputs=input_layer, units=300, \
        activation=tf.nn.relu)
    # Dropout layer1
    dropout1 = tf.layers.dropout(inputs=dense1, rate=0.4, \
        training=(mode == tf.estimator.ModeKeys.TRAIN))
    # Dense layer2
    dense2 = tf.layers.dense(inputs=dropout1, units=300, \
        activation=tf.nn.relu)
    # Dropout layer2
    dropout2 = tf.layers.dropout(inputs=dense2, rate=0.4, \
        training=(mode == tf.estimator.ModeKeys.TRAIN))
    # Dense layer3
    dense3 = tf.layers.dense(inputs=dropout2, units=300, \
        activation=tf.nn.relu)
    # Dropout layer3
    dropout3 = tf.layers.dropout(inputs=dense3, rate=0.4, \
        training=(mode == tf.estimator.ModeKeys.TRAIN))
    # Logits layer
    logits = tf.layers.dense(inputs=dropout3, units=CLASS_NUM)

    # prediction result in PREDICT and EVAL phases
    predictions = {
        # Class id
        "classes": tf.argmax(input=logits, axis=1),
        # Probabilities
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss for TRAIN and EVAL
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the training Op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARN_RATE)
        train_op = optimizer.minimize(\
            loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(\
            mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(\
            labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(\
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main():
    # Get the training and testing data from getEmbeddings
    train_data, eval_data, train_labels, eval_labels = \
        getEmbeddings("datasets/train.csv")
    train_labels = train_labels.reshape((-1, 1)).astype(np.int32)
    eval_labels = eval_labels.reshape((-1, 1)).astype(np.int32)

    # Create the Estimator
    classifier = \
        tf.estimator.Estimator(model_fn=model_fn, model_dir=tensorflow_tmp)

    # Setup logging hook for prediction
    tf.logging.set_verbosity(tf.logging.INFO)
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=200)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=50,
        num_epochs=None,
        shuffle=True)
    classifier.train(
        input_fn=train_input_fn,
        steps=TRAIN_STEP,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)   # 81.42%
    
    # Draw the confusion matrix
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        num_epochs=1,
        shuffle=False)
    predict_results = classifier.predict(input_fn=predict_input_fn)
    predict_labels = [label["classes"] for label in predict_results]
    plot_cmat(eval_labels, predict_labels)


if __name__ == "__main__":
    main()


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp_tensorflow/three_layer2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a30261a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized au

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into tmp_tensorflow/three_layer2/model.ckpt.
INFO:tensorflow:probabilities = [[0.46328693 0.53671307]
 [0.4970001  0.5029999 ]
 [0.38129651 0.61870349]
 [0.4836256  0.5163744 ]
 [0.33201242 0.66798758]
 [0.63329502 0.36670498]
 [0.30184877 0.69815123]
 [0.53907547 0.46092453]
 [0.41920058 0.58079942]
 [0.44581995 0.55418005]
 [0.53278559 0.46721441]
 [0.51902946 0.48097054]
 [0.43678221 0.56321779]
 [0.46824215 0.53175785]
 [0.62797963 0.37202037]
 [0.54615365 0.45384635]
 [0.47343325 0.52656675]
 [0.49958494 0.50041506]
 [0.32230149 0.67769851]
 [0.42446697 0.57553303]
 [

INFO:tensorflow:loss = 0.7461152672767639, step = 601 (0.433 sec)
INFO:tensorflow:global_step/sec: 216.861
INFO:tensorflow:loss = 0.7307040691375732, step = 701 (0.458 sec)
INFO:tensorflow:global_step/sec: 237.118
INFO:tensorflow:probabilities = [[0.61580011 0.38419989]
 [0.48137145 0.51862855]
 [0.48858928 0.51141072]
 [0.52222869 0.47777131]
 [0.56792062 0.43207938]
 [0.66755418 0.33244582]
 [0.5734406  0.4265594 ]
 [0.49808301 0.50191699]
 [0.40775631 0.59224369]
 [0.4968795  0.5031205 ]
 [0.41919948 0.58080052]
 [0.48972871 0.51027129]
 [0.43463724 0.56536276]
 [0.52614982 0.47385018]
 [0.49105536 0.50894464]
 [0.55069102 0.44930898]
 [0.48312621 0.51687379]
 [0.39395629 0.60604371]
 [0.45411993 0.54588007]
 [0.49783461 0.50216539]
 [0.52890194 0.47109806]
 [0.45603811 0.54396189]
 [0.68717366 0.31282634]
 [0.52078054 0.47921946]
 [0.46179929 0.53820071]
 [0.48107718 0.51892282]
 [0.48308622 0.51691378]
 [0.51132386 0.48867614]
 [0.54946616 0.45053384]
 [0.52958862 0.47041138]
 [0.

INFO:tensorflow:loss = 0.7080568671226501, step = 1801 (0.429 sec)
INFO:tensorflow:global_step/sec: 227.07
INFO:tensorflow:loss = 0.7431718707084656, step = 1901 (0.437 sec)
INFO:tensorflow:global_step/sec: 225.178
INFO:tensorflow:probabilities = [[0.52627668 0.47372332]
 [0.47802836 0.52197164]
 [0.50455239 0.49544761]
 [0.53012101 0.46987899]
 [0.38991929 0.61008071]
 [0.66314744 0.33685256]
 [0.43141601 0.56858399]
 [0.46207826 0.53792174]
 [0.48748221 0.51251779]
 [0.52165726 0.47834274]
 [0.61850097 0.38149903]
 [0.61318111 0.38681889]
 [0.4775331  0.5224669 ]
 [0.52766948 0.47233052]
 [0.44365204 0.55634796]
 [0.52687858 0.47312142]
 [0.54163721 0.45836279]
 [0.44916024 0.55083976]
 [0.54547919 0.45452081]
 [0.49270558 0.50729442]
 [0.40858985 0.59141015]
 [0.49502317 0.50497683]
 [0.48242813 0.51757187]
 [0.49780819 0.50219181]
 [0.43937775 0.56062225]
 [0.60981967 0.39018033]
 [0.54697377 0.45302623]
 [0.53187988 0.46812012]
 [0.43720687 0.56279313]
 [0.50150433 0.49849567]
 [0

INFO:tensorflow:loss = 0.6860988140106201, step = 3001 (0.335 sec)
INFO:tensorflow:global_step/sec: 284.013
INFO:tensorflow:loss = 0.6958844065666199, step = 3101 (0.350 sec)
INFO:tensorflow:global_step/sec: 297.568
INFO:tensorflow:probabilities = [[0.56566433 0.43433567]
 [0.61686682 0.38313318]
 [0.50332296 0.49667704]
 [0.43041933 0.56958067]
 [0.62386833 0.37613167]
 [0.50582987 0.49417013]
 [0.52384987 0.47615013]
 [0.57151238 0.42848762]
 [0.60946171 0.39053829]
 [0.64891828 0.35108172]
 [0.50292794 0.49707206]
 [0.53742623 0.46257377]
 [0.53890097 0.46109903]
 [0.43618084 0.56381916]
 [0.52482306 0.47517694]
 [0.42155949 0.57844051]
 [0.47546012 0.52453988]
 [0.51598305 0.48401695]
 [0.46111477 0.53888523]
 [0.49950055 0.50049945]
 [0.42637551 0.57362449]
 [0.43423778 0.56576222]
 [0.49927265 0.50072735]
 [0.53716248 0.46283752]
 [0.54731991 0.45268009]
 [0.43585489 0.56414511]
 [0.52573029 0.47426971]
 [0.52346367 0.47653633]
 [0.53471623 0.46528377]
 [0.53263791 0.46736209]
 [

...

INFO:tensorflow:loss = 0.591390073299408, step = 18601 (0.337 sec)
INFO:tensorflow:global_step/sec: 285.325
INFO:tensorflow:loss = 0.5848689675331116, step = 18701 (0.349 sec)
INFO:tensorflow:global_step/sec: 275.127
INFO:tensorflow:probabilities = [[0.44186381 0.55813619]
 [0.45954976 0.54045024]
 [0.50846187 0.49153813]
 [0.45402203 0.54597797]
 [0.59484516 0.40515484]
 [0.42914426 0.57085574]
 [0.48078959 0.51921041]
 [0.44122749 0.55877251]
 [0.40550581 0.59449419]
 [0.45162487 0.54837513]
 [0.43430005 0.56569995]
 [0.64611047 0.35388953]
 [0.37748921 0.62251079]
 [0.78648192 0.21351808]
 [0.74704982 0.25295018]
 [0.46048984 0.53951016]
 [0.47244997 0.52755003]
 [0.41923579 0.58076421]
 [0.52116451 0.47883549]
 [0.48933706 0.51066294]
 [0.6186986  0.3813014 ]
 [0.54110081 0.45889919]
 [0.49784137 0.50215863]
 [0.59806236 0.40193764]
 [0.61196189 0.38803811]
 [0.62286673 0.37713327]
 [0.59499039 0.40500961]
 [0.83407467 0.16592533]
 [0.54462723 0.45537277]
 [0.4350721  0.5649279 ]
 

INFO:tensorflow:loss = 0.5690089464187622, step = 19801 (0.359 sec)
INFO:tensorflow:global_step/sec: 280.457
INFO:tensorflow:loss = 0.6319172382354736, step = 19901 (0.355 sec)
INFO:tensorflow:Saving checkpoints for 20000 into tmp_tensorflow/three_layer2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.6093389987945557.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-08T21:56:01Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from tmp_tensorflow/three_layer2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-08-21:56:01
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.7979774, global_step = 20000, loss = 0.5819776
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: tmp_ten

{'accuracy': 0.7979774, 'loss': 0.5819776, 'global_step': 20000}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp_tensorflow/three_layer2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


/Users/mgahlot/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function plot_confusion_matrix is deprecated; This will be removed in v0.4.0. Please use scikitplot.metrics.plot_confusion_matrix instead.
  warnings.warn(msg, category=DeprecationWarning)


<Figure size 640x480 with 2 Axes>